# Fator de Visão de Céu a partir do LiDAR

Podemos também determinar o Fator de Visão de céu, ou Sky View Factor diretamente do LiDAR sem gerar um raster.
Para fins de performance podemos reduzir a densidade de pontos para cerca de 1 metro de resolução e a partir de cada ponto de Ground clacular a coordenada astronômica (Azimute e ALtura) e a distância linear para cada ponto das classes de Edificação e outras feições.

Para esse primeiro teste vamos apenas usar uma folha SCM, sabendo que inerferências para visão de céu podem se estender muito além dos limites das dimensões de uma folha SCM.

In [3]:
import pdal

In [54]:
pipeline="""{
  "pipeline": [
    {
        "type": "readers.las",
        "filename": "arquivos/MDS/MDS_3314-231.laz"
    },
    {
        "type": "filters.sample",
        "radius": 1
    }
  ]
}"""

In [55]:
r = pdal.Pipeline(pipeline)
r.validate()
r.execute()
arrays = r.arrays

In [56]:
len(arrays[0])

471962

In [57]:
arrays[0][732]

(333486.38, 7395559.83, 725.18, 5, 1, 1, 0, 0, 2, -14., 17, 6, 356569.65684247)

In [58]:
#dir(r)
r.schema

{'schema': {'dimensions': [{'name': 'X', 'size': 8, 'type': 'floating'},
   {'name': 'Y', 'size': 8, 'type': 'floating'},
   {'name': 'Z', 'size': 8, 'type': 'floating'},
   {'name': 'Intensity', 'size': 2, 'type': 'unsigned'},
   {'name': 'ReturnNumber', 'size': 1, 'type': 'unsigned'},
   {'name': 'NumberOfReturns', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanDirectionFlag', 'size': 1, 'type': 'unsigned'},
   {'name': 'EdgeOfFlightLine', 'size': 1, 'type': 'unsigned'},
   {'name': 'Classification', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanAngleRank', 'size': 4, 'type': 'floating'},
   {'name': 'UserData', 'size': 1, 'type': 'unsigned'},
   {'name': 'PointSourceId', 'size': 2, 'type': 'unsigned'},
   {'name': 'GpsTime', 'size': 8, 'type': 'floating'}]}}

## Importando dados para Pandas

In [30]:
import pandas as pd

In [59]:
df = pd.DataFrame(arrays[0])

In [60]:
df

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime
0,333499.29,7395564.35,735.34,18,1,1,0,0,6,-14.0,17,6,356569.366065
1,333499.23,7395565.79,735.41,18,1,1,0,0,6,-14.0,17,6,356569.366090
2,333499.20,7395566.79,735.25,15,1,1,0,0,6,-14.0,17,6,356569.366105
3,333499.15,7395568.21,735.37,14,1,1,0,0,6,-14.0,17,6,356569.366129
4,333499.10,7395569.68,735.30,17,1,1,0,0,6,-15.0,17,6,356569.366153
...,...,...,...,...,...,...,...,...,...,...,...,...,...
471957,333523.36,7395570.78,735.52,2,1,2,1,0,19,13.0,40,6,363044.324524
471958,333521.03,7395570.91,731.07,5,1,1,1,0,6,13.0,40,6,363044.342732
471959,333520.96,7395571.00,732.50,8,1,1,1,0,6,13.0,40,6,363044.342740
471960,333521.09,7395571.84,731.83,18,1,1,1,0,6,13.0,40,6,363044.360909


In [61]:
df.Classification.unique()

array([ 6, 20,  2, 19,  5,  9], dtype=uint8)

In [62]:
# filtrando somente os grounds == 2
df[df.Classification == 2]

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime
18,333499.25,7395550.72,724.76,6,1,1,0,0,2,-13.0,17,6,356569.383978
20,333499.14,7395553.60,724.86,5,1,1,0,0,2,-13.0,17,6,356569.384026
26,333498.88,7395560.92,724.87,5,1,1,0,0,2,-14.0,17,6,356569.384146
38,333498.72,7395562.04,725.02,36,1,1,1,0,2,-14.0,17,6,356569.386890
77,333498.36,7395551.34,724.89,17,1,1,0,0,2,-13.0,17,6,356569.402162
...,...,...,...,...,...,...,...,...,...,...,...,...,...
471680,333563.96,7395552.14,724.30,15,1,1,0,0,2,9.0,40,6,363043.837829
471723,333558.16,7395554.04,724.33,11,1,1,0,0,2,10.0,40,6,363043.892247
471739,333556.44,7395554.74,724.53,8,1,1,0,0,2,10.0,40,6,363043.910399
471792,333530.38,7395555.62,724.47,6,1,1,0,0,2,12.0,40,6,363043.964401
